In [ ]:
pip install catboost


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 9.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, RepeatedStratifiedKFold

In [ ]:
data_train=pd.read_csv('/content/train.csv')
data_train

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15643359.0,Chidubem,605.0,France,Male,29.0,5.0,0.00,1.0,1.0,1.0,151623.91,0.0
1,1,15624703.0,Okechukwu,700.0,France,Male,37.0,4.0,115301.31,2.0,1.0,1.0,162501.83,0.0
2,2,15583392.0,Okeke,572.0,France,Male,49.0,2.0,62610.96,1.0,1.0,1.0,142514.69,1.0
3,3,15594878.0,Yobanna,813.0,Germany,Male,31.0,8.0,134591.35,1.0,0.0,0.0,90591.15,0.0
4,4,15792669.0,Ch'ang,670.0,France,Female,38.0,7.0,0.00,1.0,1.0,0.0,148528.24,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14995,15594816.0,Onyemachukwu,714.0,Germany,Male,25.0,10.0,122453.97,2.0,1.0,0.0,126143.27,0.0
14996,14996,15644882.0,Yefimova,617.0,France,Male,51.0,6.0,0.00,1.0,1.0,0.0,10771.46,1.0
14997,14997,15682778.0,Onyemachukwu,748.0,Spain,Male,43.0,2.0,0.00,2.0,1.0,0.0,52411.99,0.0
14998,14998,15787229.0,Ch'ang,717.0,Spain,Male,27.0,5.0,0.00,2.0,1.0,1.0,103608.06,0.0


In [ ]:
data_test=pd.read_csv("/content/test.csv")
data_test

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,15000,15656300.0,Smith,602.0,France,Male,38.0,0.0,128524.19,2.0,0.0,1.0,78622.46
1,15001,15610454.0,Chialuka,624.0,France,Male,39.0,4.0,0.00,1.0,1.0,1.0,133086.45
2,15002,15732672.0,T'ang,744.0,Spain,Male,30.0,3.0,0.00,2.0,1.0,0.0,134732.99
3,15003,15586821.0,Hsia,714.0,France,Female,44.0,8.0,0.00,1.0,1.0,0.0,132979.14
4,15004,15711352.0,Nwokeocha,670.0,Germany,Female,25.0,9.0,134970.12,2.0,0.0,1.0,60420.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,24995,15807900.0,Ugoji,602.0,France,Male,25.0,5.0,0.00,2.0,0.0,1.0,75562.47
9996,24996,15655590.0,Chien,739.0,Germany,Female,45.0,0.0,81710.46,1.0,0.0,1.0,103557.85
9997,24997,15627854.0,Kaur,744.0,Spain,Male,35.0,2.0,0.00,2.0,1.0,0.0,106920.57
9998,24998,15796569.0,Toscani,564.0,France,Female,34.0,0.0,0.00,1.0,1.0,0.0,155876.06


In [ ]:
data_submit=pd.read_csv("/content/sample_submission.csv")
data_submit

,id,Exited
0,15000,0.5
1,15001,0.5
2,15002,0.5
3,15003,0.5
4,15004,0.5
...,...,...
9995,24995,0.5
9996,24996,0.5
9997,24997,0.5
9998,24998,0.5


In [ ]:
data_train.nunique()

id                 15000
CustomerId          6349
Surname              760
CreditScore          376
Geography              3
Gender                 2
Age                   55
Tenure                11
Balance             3394
NumOfProducts          4
HasCrCard              2
IsActiveMember         2
EstimatedSalary     6166
Exited                 2
dtype: int64

In [ ]:
data_train.isnull().sum()

id                 0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [ ]:
data_train.describe()

,id,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,15000.000000,1.500000e+04,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,1.500000e+04,15000.000000
mean,7499.500000,1.569298e+07,659.446933,37.732267,4.976267,43502.827963,1.586533,0.780133,0.498867,1.178744e+05,0.204800
std,4330.271354,7.149846e+04,73.272078,8.193802,2.800421,60245.054927,0.532163,0.414170,0.500015,4.756063e+04,0.403569
min,0.000000,1.556570e+07,438.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,4.477300e+02,0.000000
25%,3749.750000,1.563518e+07,604.000000,32.000000,3.000000,0.000000,1.000000,1.000000,0.000000,8.308433e+04,0.000000
50%,7499.500000,1.569074e+07,663.000000,37.000000,5.000000,0.000000,2.000000,1.000000,0.000000,1.225805e+05,0.000000
75%,11249.250000,1.575798e+07,710.000000,42.000000,7.000000,111204.467500,2.000000,1.000000,1.000000,1.568682e+05,0.000000
max,14999.000000,1.581566e+07,850.000000,74.000000,10.000000,187911.550000,4.000000,1.000000,1.000000,1.816018e+06,1.000000


In [ ]:
X_train = data_train.drop(columns=['Exited', 'id', 'CustomerId', 'Surname'])
y_train = data_train['Exited']
X_test = data_test.drop(columns=['id', 'CustomerId', 'Surname'])

In [ ]:
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

In [ ]:
X_train, X_test = X_train.align(X_test, join='outer', axis=1, fill_value=0)

# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
X_train_final, X_val, y_train_final, y_val = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=42)

# Define the model
model = Sequential([
    Dense(11, activation='relu', input_dim=X_train_scaled.shape[1]),  # Input dimension should match the number of features
    Dense(11, activation='relu'),
    Dense(11, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train_final, y_train_final, epochs=100, validation_data=(X_val, y_val))

# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)

Epoch 1/100
375/375 [==============================] - 3s 4ms/step - loss: 0.4334 - accuracy: 0.8179 - val_loss: 0.3451 - val_accuracy: 0.8557
Epoch 2/100
375/375 [==============================] - 3s 8ms/step - loss: 0.3363 - accuracy: 0.8611 - val_loss: 0.3160 - val_accuracy: 0.8683
Epoch 3/100
375/375 [==============================] - 2s 6ms/step - loss: 0.3186 - accuracy: 0.8682 - val_loss: 0.3006 - val_accuracy: 0.8727
Epoch 4/100
375/375 [==============================] - 2s 4ms/step - loss: 0.3082 - accuracy: 0.8715 - val_loss: 0.2912 - val_accuracy: 0.8777
Epoch 5/100
375/375 [==============================] - 2s 5ms/step - loss: 0.2999 - accuracy: 0.8762 - val_loss: 0.2839 - val_accuracy: 0.8793
Epoch 6/100
375/375 [==============================] - 2s 5ms/step - loss: 0.2913 - accuracy: 0.8766 - val_loss: 0.2793 - val_accuracy: 0.8823
Epoch 7/100
375/375 [==============================] - 2s 6ms/step - loss: 0.2845 - accuracy: 0.8821 - val_loss: 0.2702 - val_accuracy: 0.8863

In [18]:
predictions = model.predict(X_test_scaled)

# Create DataFrame with Id and Predicted Probabilities
submission_df = pd.DataFrame({'id': data_test['id'], 'Exited': predictions.flatten()})

# Save DataFrame to CSV
submission_df.to_csv('submission1.csv', index=False)

313/313 [==============================] - 1s 3ms/step
